# Gradient Descent Applied To Linear Regression

#### Sources:

http://ufldl.stanford.edu/tutorial/supervised/LinearRegression/
https://www.youtube.com/watch?v=YS_EztqZCD8
https://introml.mit.edu/notes/gradient_descent.html

#### Introduction

Gradient descent is a method used to train a variety of machine learning algorithms. In this notebook, we will use a few common gradient descent varients to perform linear regression, and explore how gradient descent works. Although linear regression has a closed form solution (I go through that here [MultipleLinear+PolynomialRegression](https://github.com/JakeLarimer/MLAlgosFromScratch/blob/main/MultipleLinearRegression%2BPolynomialRegression.ipynb)), other algorithms often do not, requiring gradient descent to learn. Given the simplicity of linear regression compared to other algorithms, it acts as a good vessel to demonstrate the application of gradient descent.

The ovarching goal of gradient descent is to identify model parameters that lead to a local minima of a "cost" or "loss" function. These functions essentially defines the error between the predicted values from a model, and the actual values. Loss and cost functions are similar, but have an important distinction. Loss refers to the sum of the function over all predicted and true values, while cost is the average error per value. Different algorithms use different loss functions. For example with linear regression, we use the $\textit{sum of squared errors}$ or $\textit{SSE}$:

$$
\sum_{i=1}^n(y_i-\hat{y}_i)^2
$$

where $y_i$ is a true observation and $\hat{y}_i$ is the corresponing predicted value.

Loss functions are generally denoted as $J(\Theta)$ where $\Theta$ is a vector containing the model parameters. Thus, our goal is to find $\Theta^*$ such that $\Theta^*=\arg \min_{\Theta}J(\Theta)$. With gradient descent, 

In [1]:
import numpy as np
import matplotlib.pyplot as plt